<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

В данном проекте решается задача бинарной классификации

## Подготовка

испорт библиотек

In [1]:
import numpy as np
import pandas as pd
import re
from pymystem3 import Mystem
from nltk.corpus import stopwords as nltk_stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostRegressor
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline

In [2]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Загрузим датасет и посмотрим, что в нем

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.sample(5)

,Unnamed: 0,text,toxic
94611,94703,"""\n\n WP:NPA \n\nSarumio, making edit summarie...",0
120705,120810,".\n\nSecondly, it has nothing to do with the d...",0
141433,141586,"""\n\nMy civility\nYou sir have not had said IF...",0
15472,15489,"""\n\n The phonology of Urdu appears to be diff...",0
93566,93658,Kinky FAC \n\nI'll be along asap. Sorry for de...,0


In [6]:
df.tail()

,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Посмотрим на баланс классов

In [8]:
len(df[df['toxic'] == 1])

16186

In [9]:
len(df[df['toxic'] == 0])

143106

Сильный дисбаланс, при обучении нужно это учитывать

In [10]:
df.drop('Unnamed: 0', axis=1, inplace=True)

Избавился от лишнего столбца

Расчитстим текст

In [11]:
def clear_text(text):
    
    f = re.sub(r"[^a-zA-Z']", ' ', text)
    f = f.split()
    lemm_text = " ".join(f)
        
    return lemm_text

In [12]:
df['text'] = df['text'].apply(clear_text)

И проведем лемматизацию

In [13]:

def lemmatize_with_wordnet(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in nltk.pos_tag(tokens)]
    lemm_text = " ".join(lemmatized_tokens)
    return lemm_text

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN



tqdm.pandas()  
df['lemm_text'] = df['text'].progress_apply(lemmatize_with_wordnet)







  0%|          | 0/159292 [00:00<?, ?it/s]

In [14]:
df.head()

,text,toxic,lemm_text
0,Explanation Why the edits made under my userna...,0,Explanation Why the edits make under my userna...
1,D'aww He matches this background colour I'm se...,0,D'aww He match this background colour I 'm see...
2,Hey man I'm really not trying to edit war It's...,0,Hey man I 'm really not try to edit war It 's ...
3,More I can't make any real suggestions on impr...,0,More I ca n't make any real suggestion on impr...
4,You sir are my hero Any chance you remember wh...,0,You sir be my hero Any chance you remember wha...


In [15]:
stopwords = set(nltk_stopwords.words('english'))

**Итог**

Привели текстовые данные в порядок. Так же был обнаружен дисбаланс классов, при обучении моделей мы передадим им аргумент соответсвующий исправлению этой ситуации

## Обучение

Создадим признкаки и выделим целевой признак для обучения

In [16]:
features = df['text']
target=df['toxic'].values

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = .25, random_state = 12345)


X_train.shape[0], X_test.shape[0]

(119469, 39823)

**TF-IDF**

In [17]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tfidf_train = count_tf_idf.fit_transform(X_train)
tfidf_test = count_tf_idf.transform(X_test)

После преобразования текста в векторный вид, можно начать обучения.

Начнем с Логистической регрессии

In [19]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Этап векторизации
    ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))  # Модель
])

# Задание сетки параметров
param_grid = {
    'tfidf__max_df': [0.75, 1.0],
    'model__C': [1, 10],
    'model__penalty': ['l1', 'l2']
}

# Создание объекта GridSearchCV с пайплайном и параметрами
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=3, verbose=True, n_jobs=-1)

# Применение GridSearchCV
grid_search.fit(X_train, y_train)

# Вывод результатов
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best parameters: {'model__C': 10, 'model__penalty': 'l1', 'tfidf__max_df': 0.75}
Best score: 0.7758452496739876


Отличный резултат, удолетворяющий требования(метрика f1 не менее 0.75)

Посмотрим, как справляется случайный лес

In [26]:
# Создание пайплайна с векторизацией и моделью
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Этап векторизации
    ('model', RandomForestClassifier(random_state=12345, class_weight='balanced'))  # Модель
])

# Задание сетки параметров для RandomForestClassifier
param_grid_rf = {
    'tfidf__max_df': [0.75, 1.0],
    'model__n_estimators': [10, 30, 100],
    'model__max_depth': [1, 10, 20],
}

# Создание объекта GridSearchCV с пайплайном и параметрами
grid_search_rf = GridSearchCV(pipeline, param_grid_rf, scoring='f1', cv=3, verbose=True, n_jobs=-1)

# Применение GridSearchCV
grid_search_rf.fit(X_train, y_train)

# Вывод результатов
print("Best parameters:", grid_search_rf.best_params_)
print("Best score:", grid_search_rf.best_score_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
Best parameters: {'model__max_depth': 20, 'model__n_estimators': 100, 'tfidf__max_df': 0.75}
Best score: 0.3713053894839537


Результат дерева нас не устравивает 

In [22]:

y_pred = grid_search.predict(X_test)


f1 = f1_score(y_test, y_pred)
print("F1 Score on test set:", f1)

F1 Score on test set: 0.7875647668393783


## Выводы

В ходе обчения моделей было выявлено, что логистическая регрессия отлично справилась с условием проекта и выполняет неплохой классификацию текстов по наличию токсичности в комментарии